In [1]:
# !pip install --upgrade pip
# !pip install -U scikit-image
# !pip install -U scikit-learn

In [2]:
import os
import numpy as np
import pandas as pd

from skimage.transform import resize
from skimage.io import imread

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, classification_report

In [5]:
### Train data ###
train_data_lst = []
train_target_lst = []

train_dir_lst = ['/Spoof_data/Training Biometrika Live/live', 
                 '/Spoof_data/Training Biometrika Spoof/Training Biometrika Spoof/spoof']

for idx, train_dir in enumerate(train_dir_lst):
    for img in os.listdir(train_dir):
#         print(idx, '---', os.path.join(train_dir, img))
        img_array = imread(os.path.join(train_dir, img))
        img_resized = resize(img_array, (150, 150, 3))
        train_data_lst.append(img_resized.flatten())
        train_target_lst.append(idx)
        
train_flat_data = np.array(train_data_lst)
train_target = np.array(train_target_lst)
train_df = pd.DataFrame(train_flat_data)
train_df['Target'] = train_target
train_df = train_df.sample(frac=1).reset_index(drop=True)
print(train_df.shape)

train_x = train_df.iloc[:,:-1]
train_y = train_df.iloc[:,-1]
print(train_x.shape, train_y.shape)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './Spoof_data/Training Biometrika Live/live'

In [4]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.266816,0.266816,0.266816,0.268971,0.268971,0.268971,0.267299,0.267299,0.267299,0.272649,...,0.433493,0.433493,0.433493,0.459405,0.459405,0.459405,0.550058,0.550058,0.550058,1
1,0.285777,0.285777,0.285777,0.285970,0.285970,0.285970,0.285139,0.285139,0.285139,0.293338,...,0.482585,0.482585,0.482585,0.476402,0.476402,0.476402,0.555687,0.555687,0.555687,0
2,0.258346,0.258346,0.258346,0.254828,0.254828,0.254828,0.263054,0.263054,0.263054,0.266147,...,0.443288,0.443288,0.443288,0.440373,0.440373,0.440373,0.503867,0.503867,0.503867,0
3,0.280049,0.280049,0.280049,0.280728,0.280728,0.280728,0.282636,0.282636,0.282636,0.280138,...,0.439998,0.439998,0.439998,0.460962,0.460962,0.460962,0.543188,0.543188,0.543188,1
4,0.301422,0.301422,0.301422,0.301511,0.301511,0.301511,0.298038,0.298038,0.298038,0.304883,...,0.512512,0.512512,0.512512,0.510083,0.510083,0.510083,0.599894,0.599894,0.599894,1


In [5]:
### Test data ###
test_data_lst = []
test_target_lst = []

test_dir_lst = ['./Spoof_data/Testing Biometrika Live/live', 
                 './Spoof_data/Testing Biometrika Spoof/Testing Biometrika Spoof/spoof']

for idx, test_dir in enumerate(test_dir_lst):
    for img in os.listdir(test_dir):
#         print(idx, '---', os.path.join(test_dir, img))
        img_array = imread(os.path.join(test_dir, img))
        img_resized = resize(img_array, (150, 150, 3))
        test_data_lst.append(img_resized.flatten())
        test_target_lst.append(idx)
        
test_flat_data = np.array(test_data_lst)
test_target = np.array(test_target_lst)
test_df = pd.DataFrame(test_flat_data)
test_df['Target'] = test_target
test_df = test_df.sample(frac=1).reset_index(drop=True)
print(test_df.shape)

test_x = test_df.iloc[:,:-1]
test_y = test_df.iloc[:,-1]
print(test_x.shape, test_y.shape)

(400, 67501)
(400, 67500) (400,)


In [6]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.262837,0.262837,0.262837,0.261944,0.261944,0.261944,0.260291,0.260291,0.260291,0.265083,...,0.429728,0.429728,0.429728,0.409868,0.409868,0.409868,0.500587,0.500587,0.500587,0
1,0.267935,0.267935,0.267935,0.270116,0.270116,0.270116,0.266632,0.266632,0.266632,0.274519,...,0.466724,0.466724,0.466724,0.457790,0.457790,0.457790,0.516049,0.516049,0.516049,0
2,0.274791,0.274791,0.274791,0.274735,0.274735,0.274735,0.276969,0.276969,0.276969,0.284049,...,0.452446,0.452446,0.452446,0.462730,0.462730,0.462730,0.536895,0.536895,0.536895,0
3,0.262543,0.262543,0.262543,0.261048,0.261048,0.261048,0.264334,0.264334,0.264334,0.267360,...,0.436984,0.436984,0.436984,0.423587,0.423587,0.423587,0.508081,0.508081,0.508081,0
4,0.267368,0.267368,0.267368,0.269940,0.269940,0.269940,0.266399,0.266399,0.266399,0.272645,...,0.442543,0.442543,0.442543,0.456009,0.456009,0.456009,0.534258,0.534258,0.534258,0


In [7]:
# Train data split into training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(train_x, 
                                                    train_y, 
                                                    test_size=0.25, 
                                                    random_state=77, 
                                                    stratify=train_y
                                                   )
print('Train:', x_train.shape, y_train.shape)
print('Test: ', x_test.shape, y_test.shape)

Train: (305, 67500) (305,)
Test:  (102, 67500) (102,)


# SVM model (base)

In [8]:
base_model = svm.SVC(probability=True)
base_model.fit(x_train, y_train)
print(base_model.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [27]:
print(base_model.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


### Prediction on the split validation dataset

In [9]:
y_pred = base_model.predict(x_test)
base_accuracy = round(accuracy_score(y_pred, y_test)*100, 3)
print('Accuracy Score: ', base_accuracy)
base_precision = round(precision_score(y_pred, y_test)*100, 3)
print('Precision Score:', base_precision)

Accuracy Score:  94.118
Precision Score: 94.231


In [10]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        50
           1       0.94      0.94      0.94        52

    accuracy                           0.94       102
   macro avg       0.94      0.94      0.94       102
weighted avg       0.94      0.94      0.94       102



### Prediction on the given test samples

In [21]:
pred_y = base_model.predict(test_x)
base_accuracy_test = round(accuracy_score(pred_y, test_y)*100, 3)
print('Accuracy Score: ', base_accuracy_test)
base_precision_test = round(precision_score(pred_y, test_y)*100, 3)
print('Precision Score:', base_precision_test)

Accuracy Score:  88.5
Precision Score: 99.5


In [12]:
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.99      0.78      0.87       200
           1       0.82      0.99      0.90       200

    accuracy                           0.89       400
   macro avg       0.90      0.89      0.88       400
weighted avg       0.90      0.89      0.88       400



# SVM model using GridSearchCV 

In [13]:
param_grid = {'C': [0.1, 1, 10],
              'kernel': ['rbf', 'poly'],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None],
             }
svc = svm.SVC(probability=True)  # verbose=10
grid_search_model = GridSearchCV(svc, param_grid)
grid_search_model.fit(x_train, y_train)
print()
print(grid_search_model.best_params_)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]
{'C': 10, 'class_weight': 'bala

In [25]:
print(grid_search_model.best_params_)

{'C': 10, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}


### Prediction on the split validation dataset

In [14]:
y_pred = grid_search_model.predict(x_test)
grid_accuracy = round(accuracy_score(y_pred, y_test)*100, 3)
print('Accuracy Score: ', grid_accuracy)
grid_precision = round(precision_score(y_pred, y_test)*100, 3)
print('Precision Score:', grid_precision)

Accuracy Score:  94.118
Precision Score: 92.308


In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94        50
           1       0.96      0.92      0.94        52

    accuracy                           0.94       102
   macro avg       0.94      0.94      0.94       102
weighted avg       0.94      0.94      0.94       102



### Prediction on the given test samples

In [20]:
pred_y = grid_search_model.predict(test_x)
grid_accuracy_test = round(accuracy_score(pred_y, test_y)*100, 3)
print('Accuracy Score: ', grid_accuracy_test)
grid_precision_test = round(precision_score(pred_y, test_y)*100, 3)
print('Precision Score:', grid_precision_test)

Accuracy Score:  90.5
Precision Score: 99.5


In [17]:
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.99      0.81      0.90       200
           1       0.84      0.99      0.91       200

    accuracy                           0.91       400
   macro avg       0.92      0.91      0.90       400
weighted avg       0.92      0.91      0.90       400

